In [244]:
import pandas as pd 

#import file

File = pd.read_csv("Text_Similarity_Dataset.csv") 
T1=File['text1']
T2=File['text2']
Id=File['Unique_ID']

Token1=[]
Token2=[]
import re
import nltk
from nltk.corpus import stopwords

for i in range(len(T1)):
    # Sentence conveerted into lower case 
    
    p1 = T1[i].lower()
    p2 = T2[i].lower()
    
    # Remove non letter and white space characters
    
    p1 = re.sub('[^a-zA-Z]', ' ', p1)
    p2 = re.sub('[^a-zA-Z]', ' ', p2)
    p1 = re.sub(r'\s+', ' ', p1)
    p2 = re.sub(r'\s+', ' ', p2)

    # Sentence tokenization
    
    all1 = nltk.sent_tokenize(p1)
    all2 = nltk.sent_tokenize(p2)

    all_words1 = [nltk.word_tokenize(sent) for sent in all1]
    all_words2 = [nltk.word_tokenize(sent) for sent in all2]
    
    # Stop words tokes are removed 
    
    for y in range(len(all_words1)):
        all_words1[y] = [w for w in all_words1[y] if w not in stopwords.words('english')]
    for y in range(len(all_words2)):
        all_words2[y] = [w for w in all_words2[y] if w not in stopwords.words('english')]
    
    Token1+=all_words1
    Token2+=all_words2
    



In [245]:
#Corpus is generated from sentences
nCorpus=[]

for i in range(len(Token1)):
    nCorpus.append(Token1[i])
    nCorpus.append(Token2[i])

# Corpus fed into Fastext with default parameters and min_count=1
from gensim.models import FastText

model = FastText(nCorpus, min_count=1)

# Return a dictionary with unique words and count of words
def counter(slist):
    dict={}
    for y in slist:
        if y not in dict.keys():
            dict[y]=1
        else:
            dict[y]+=1
    return dict

# Calculates mean of a given dictionary by converting the keys into vectors using fasttext model and taking sum.
import numpy as np
def centroid(dict,len):
    sum = np.zeros(100)
    for y in dict.keys():
        sum+=(dict[y]*model.wv[y])
    return(sum/len)

# Sentences are converted into dictionary with word count
Tokendict1=[]
Tokendict2=[]
for i in range(len(Token1)):
    Tokendict1.append(counter(Token1[i]))
    Tokendict2.append(counter(Token2[i]))

# cosine between two sence pairs are stored
from scipy import spatial

cosine=[]
for i in range(len(Token1)):
    cosine.append(1 - spatial.distance.cosine(centroid(Tokendict1[i],len(Token1[i])), centroid(Tokendict2[i],len(Token2[i]))))

#cosine_similarity_score.csv file is generated
import pandas as pd

data = {'Unique_ID': Id,
        'Similarity_Score': cosine
        }

df = pd.DataFrame(data, columns= ['Unique_ID', 'Similarity_Score'])

df.to_csv ('cosine_similarity_score.csv', index = False, header=True)